# 🚀 InsightSpike-AI: Foundational Experiment
## Intrinsic Motivation (ΔGED × ΔIG) Effectiveness Validation

This notebook validates the effectiveness of intrinsic motivation rewards using the combination of Graph Edit Distance changes (ΔGED) and Information Gain changes (ΔIG) in simple reinforcement learning environments.

### Experimental Design
- **Environments**: Grid-World maze & MountainCar  
- **Ablation Study**: Compare ΔGED=0, ΔIG=0, and full ΔGED×ΔIG conditions
- **Metrics**: Success rate, episode count, sample efficiency, learning curves

### Expected Outcomes
We hypothesize that agents using the full intrinsic motivation (ΔGED × ΔIG) will show:
1. Higher success rates
2. Better sample efficiency  
3. More stable learning curves
4. Faster convergence to optimal policies

In [ ]:
# Initial Colab Environment Setup
import sys
import os
from pathlib import Path

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("🔧 Running in Google Colab")
except:
    IN_COLAB = False
    print("🔧 Running in local environment")

# Install required packages for Colab
if IN_COLAB:
    print("📦 Installing required packages...")
    # Install compatible versions to avoid meta tensor issues
    !pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cpu
    !pip install sentence-transformers==2.7.0  # Compatible with PyTorch 2.2.2
    !pip install numpy==1.26.4  # Avoid NumPy 2.x compatibility issues
    !pip install gymnasium[classic_control]  # Modern replacement for gym
    !pip install seaborn pandas matplotlib
    !pip install plotly kaleido
    !pip install networkx scikit-learn
    print("✅ Package installation complete")

In [ ]:
# Import and Setup InsightSpike-AI
if IN_COLAB:
    # Get GitHub token from Colab secrets for private repository
    from google.colab import userdata
    
    try:
        github_token = userdata.get('GITHUB_TOKEN')
        print("✅ GitHub token found in secrets")
    except Exception as e:
        print("❌ GitHub token not found in secrets.")
        print("📝 Please add GITHUB_TOKEN to Colab secrets:")
        print("   1. Click the key icon (🔑) in the left sidebar")
        print("   2. Add new secret: Name='GITHUB_TOKEN', Value='your_github_token'")
        print("   3. Get token from: https://github.com/settings/tokens")
        raise e
    
    # Clone the private repository
    print("📥 Cloning InsightSpike-AI repository...")
    clone_url = f"https://{github_token}@github.com/miyauchikazuyoshi/InsightSpike-AI.git"
    !git clone $clone_url
    
    # Change to project directory
    os.chdir('/content/InsightSpike-AI')
    sys.path.append('/content/InsightSpike-AI')
    print("📁 Changed to project directory")
else:
    # Assume we're already in the project directory
    print("📁 Using local project directory")

# Verify we can import the experiment module
try:
    sys.path.append('experiments/colab_experiments/foundational_experiment')
    from intrinsic_motivation_experiment import (
        run_foundational_experiment,
        SimpleGridWorld,
        IntrinsicMotivationAgent,
        run_grid_world_experiment,
        run_mountain_car_experiment,
        create_visualization
    )
    print("✅ Successfully imported experiment modules")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("📋 Available files:")
    !ls -la experiments/colab_experiments/foundational_experiment/

In [ ]:
# Import Required Libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import torch
import warnings
from datetime import datetime
from IPython.display import display, HTML

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

print("🎯 Environment setup complete!")
print(f"📊 NumPy version: {np.__version__}")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")

## 🧪 Experiment Configuration

Let's first understand our experimental setup:

### Agent Configurations
1. **Full (ΔGED × ΔIG)**: Complete intrinsic motivation system
2. **No GED (ΔIG only)**: Information gain only
3. **No IG (ΔGED only)**: Graph edit distance only  
4. **Baseline (No intrinsic)**: Standard RL without intrinsic motivation

### Environment Details
- **Grid-World**: 8x8 grid with randomly placed obstacles
- **MountainCar**: Classic control problem requiring exploration

In [ ]:
# Quick Environment Test
print("🎮 Testing Grid-World Environment...")

# Create and test a simple Grid-World
env = SimpleGridWorld(size=6, num_obstacles=3)
state = env.reset()

print(f"✅ Environment created successfully")
print(f"📏 State space size: {env.state_space_size}")
print(f"🎯 Action space size: {env.action_space_size}")
print(f"🏁 Start position: {env.start_pos}")
print(f"🎯 Goal position: {env.goal_pos}")

# Visualize the grid
grid_viz = env.grid.copy()
grid_viz[env.current_pos] = 0.5  # Current position

plt.figure(figsize=(6, 6))
plt.imshow(grid_viz, cmap='RdYlBu', interpolation='nearest')
plt.title('Sample Grid-World Environment')
plt.colorbar(label='Cell Type (-1: Obstacle, 0: Empty, 0.5: Agent, 1: Goal)')
plt.show()

In [ ]:
# Test Agent Creation
print("🤖 Testing Intrinsic Motivation Agent...")

# Create agents with different configurations
configs = [
    {"name": "Full (ΔGED × ΔIG)", "use_ged": True, "use_ig": True},
    {"name": "No GED (ΔIG only)", "use_ged": False, "use_ig": True},
    {"name": "No IG (ΔGED only)", "use_ged": True, "use_ig": False},
    {"name": "Baseline (No intrinsic)", "use_ged": False, "use_ig": False}
]

agents = {}
for config in configs:
    agent = IntrinsicMotivationAgent(
        state_size=env.state_space_size,
        action_size=env.action_space_size,
        use_ged=config["use_ged"],
        use_ig=config["use_ig"]
    )
    agents[config["name"]] = agent
    print(f"✅ Created agent: {config['name']}")

print(f"\n🎯 Created {len(agents)} agent configurations")

## 🚀 Running the Main Experiment

Now let's run the comprehensive experiment across both environments with all agent configurations.

**Note**: This may take several minutes to complete depending on the computational resources available.

In [ ]:
# Run the Complete Foundational Experiment
print("🚀 Starting Foundational Experiment...")
print("⏰ This may take 5-10 minutes to complete...")

# Configure experiment parameters for Colab (reduced for faster execution)
if IN_COLAB:
    grid_episodes = 200  # Reduced for Colab
    grid_trials = 2
    mountain_episodes = 150
    mountain_trials = 2
else:
    grid_episodes = 300
    grid_trials = 3
    mountain_episodes = 200
    mountain_trials = 2

print(f"📊 Grid-World: {grid_episodes} episodes × {grid_trials} trials")
print(f"🏔️ MountainCar: {mountain_episodes} episodes × {mountain_trials} trials")

# Run Grid-World experiment
print("\n📊 Running Grid-World Experiment...")
grid_results = run_grid_world_experiment(episodes=grid_episodes, trials=grid_trials)

print("✅ Grid-World experiment completed!")
for config, results in grid_results.items():
    mean_success = np.mean(results["success_rates"])
    print(f"  {config}: {mean_success:.3f} success rate")

In [ ]:
# Run MountainCar Experiment (with error handling)
print("\n🏔️ Running MountainCar Experiment...")

try:
    mountain_car_results = run_mountain_car_experiment(
        episodes=mountain_episodes, 
        trials=mountain_trials
    )
    
    if mountain_car_results:
        print("✅ MountainCar experiment completed!")
        for config, results in mountain_car_results.items():
            mean_success = np.mean(results["success_rates"])
            print(f"  {config}: {mean_success:.3f} success rate")
    else:
        print("⚠️ MountainCar experiment returned empty results")
        mountain_car_results = None
        
except Exception as e:
    print(f"❌ MountainCar experiment failed: {e}")
    print("📊 Continuing with Grid-World results only")
    mountain_car_results = None

## 📈 Results Visualization and Analysis

Let's create comprehensive visualizations of our experimental results to understand the effectiveness of intrinsic motivation.

In [ ]:
# Create Comprehensive Visualization
print("📈 Creating comprehensive visualizations...")

# Generate the main visualization
fig = create_visualization(grid_results, mountain_car_results)

# Display the plot
plt.show()

# Create additional detailed analysis plots
fig2, axes = plt.subplots(2, 2, figsize=(15, 10))
fig2.suptitle('Detailed Analysis: Intrinsic Motivation Components', fontsize=14)

# 1. Success Rate Comparison (Detailed)
ax1 = axes[0, 0]
configs = list(grid_results.keys())
success_means = [np.mean(grid_results[config]["success_rates"]) for config in configs]
success_stds = [np.std(grid_results[config]["success_rates"]) for config in configs]

bars = ax1.bar(range(len(configs)), success_means, yerr=success_stds, 
               capsize=5, alpha=0.7, color=sns.color_palette("husl", len(configs)))
ax1.set_title('Grid-World Success Rates with Error Bars')
ax1.set_ylabel('Success Rate')
ax1.set_xticks(range(len(configs)))
ax1.set_xticklabels([c.replace(" (", "\n(") for c in configs], fontsize=9)

# Add value labels on bars
for i, (mean, std) in enumerate(zip(success_means, success_stds)):
    ax1.text(i, mean + std + 0.01, f'{mean:.3f}', ha='center', va='bottom', fontweight='bold')

# 2. Sample Efficiency Heatmap
ax2 = axes[0, 1]
efficiency_matrix = []
for config in configs:
    efficiency_matrix.append(grid_results[config]["sample_efficiency"])

efficiency_df = pd.DataFrame(efficiency_matrix, 
                           index=[c.split(" (")[0] for c in configs],
                           columns=[f"Trial {i+1}" for i in range(len(efficiency_matrix[0]))])

sns.heatmap(efficiency_df, annot=True, fmt='.3f', cmap='viridis', ax=ax2)
ax2.set_title('Sample Efficiency by Configuration and Trial')

# 3. Learning Progress Comparison
ax3 = axes[1, 0]
for i, config in enumerate(configs):
    learning_curves = grid_results[config]["learning_curves"]
    if learning_curves:
        # Calculate moving average for smoother curves
        window = 20
        for curve in learning_curves:
            if len(curve) >= window:
                smoothed = pd.Series(curve).rolling(window=window).mean()
                ax3.plot(smoothed, alpha=0.3, color=sns.color_palette("husl", len(configs))[i])
        
        # Plot mean curve
        max_length = max(len(curve) for curve in learning_curves)
        padded_curves = [curve + [curve[-1]] * (max_length - len(curve)) for curve in learning_curves]
        mean_curve = np.mean(padded_curves, axis=0)
        smoothed_mean = pd.Series(mean_curve).rolling(window=window).mean()
        ax3.plot(smoothed_mean, label=config, linewidth=2, 
                color=sns.color_palette("husl", len(configs))[i])

ax3.set_title('Learning Curves (Smoothed)')
ax3.set_xlabel('Episode')
ax3.set_ylabel('Reward')
ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# 4. Statistical Significance Test
ax4 = axes[1, 1]
from scipy import stats

# Perform pairwise t-tests between configurations
significance_matrix = np.zeros((len(configs), len(configs)))
p_values = {}

for i, config1 in enumerate(configs):
    for j, config2 in enumerate(configs):
        if i != j:
            data1 = grid_results[config1]["success_rates"]
            data2 = grid_results[config2]["success_rates"]
            _, p_value = stats.ttest_ind(data1, data2)
            significance_matrix[i, j] = p_value
            p_values[f"{config1} vs {config2}"] = p_value

# Create significance heatmap
significance_df = pd.DataFrame(significance_matrix,
                             index=[c.split(" (")[0] for c in configs],
                             columns=[c.split(" (")[0] for c in configs])

im = ax4.imshow(significance_matrix, cmap='RdYlGn_r', vmin=0, vmax=0.1)
ax4.set_title('Statistical Significance (p-values)')
ax4.set_xticks(range(len(configs)))
ax4.set_yticks(range(len(configs)))
ax4.set_xticklabels([c.split(" (")[0] for c in configs], rotation=45)
ax4.set_yticklabels([c.split(" (")[0] for c in configs])

# Add text annotations
for i in range(len(configs)):
    for j in range(len(configs)):
        if i != j:
            text = ax4.text(j, i, f'{significance_matrix[i, j]:.3f}',
                          ha="center", va="center", color="black", fontsize=8)

plt.colorbar(im, ax=ax4, label='p-value')
plt.tight_layout()
plt.show()

In [ ]:
# Statistical Analysis and Summary
print("📊 Statistical Analysis Summary")
print("=" * 50)

# Calculate effect sizes (Cohen's d)
def cohens_d(group1, group2):
    """Calculate Cohen's d effect size"""
    n1, n2 = len(group1), len(group2)
    pooled_std = np.sqrt(((n1 - 1) * np.var(group1, ddof=1) + 
                         (n2 - 1) * np.var(group2, ddof=1)) / (n1 + n2 - 2))
    return (np.mean(group1) - np.mean(group2)) / pooled_std

# Compare Full vs Baseline
full_success = grid_results["Full (ΔGED × ΔIG)"]["success_rates"]
baseline_success = grid_results["Baseline (No intrinsic)"]["success_rates"]

improvement = (np.mean(full_success) - np.mean(baseline_success)) / np.mean(baseline_success) * 100
effect_size = cohens_d(full_success, baseline_success)
_, p_value = stats.ttest_ind(full_success, baseline_success)

print(f"\n🎯 Full Intrinsic Motivation vs Baseline:")
print(f"   Improvement: {improvement:.1f}%")
print(f"   Effect Size (Cohen's d): {effect_size:.3f}")
print(f"   Statistical Significance (p-value): {p_value:.4f}")

if p_value < 0.05:
    significance = "Statistically Significant ✅"
else:
    significance = "Not Statistically Significant ❌"
print(f"   {significance}")

# Effect size interpretation
if abs(effect_size) < 0.2:
    effect_desc = "Small"
elif abs(effect_size) < 0.5:
    effect_desc = "Medium"
elif abs(effect_size) < 0.8:
    effect_desc = "Large"
else:
    effect_desc = "Very Large"

print(f"   Effect Size Interpretation: {effect_desc}")

# Summary table
print("\n📋 Complete Results Summary:")
print("-" * 70)
print(f"{'Configuration':<25} {'Success Rate':<15} {'Std Dev':<10} {'Sample Eff.':<12}")
print("-" * 70)

for config in configs:
    mean_success = np.mean(grid_results[config]["success_rates"])
    std_success = np.std(grid_results[config]["success_rates"])
    mean_efficiency = np.mean(grid_results[config]["sample_efficiency"])
    
    print(f"{config:<25} {mean_success:<15.3f} {std_success:<10.3f} {mean_efficiency:<12.3f}")

print("-" * 70)

## 💾 Save Results and Download

Let's save our experimental results and create downloadable files for further analysis.

In [ ]:
# Save Experimental Results
print("💾 Saving experimental results...")

# Create results directory
results_dir = Path("foundational_experiment_results")
results_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Prepare comprehensive results data
results_data = {
    "timestamp": timestamp,
    "experiment_type": "foundational_intrinsic_motivation",
    "parameters": {
        "grid_world_episodes": grid_episodes,
        "grid_world_trials": grid_trials,
        "mountain_car_episodes": mountain_episodes if mountain_car_results else 0,
        "mountain_car_trials": mountain_trials if mountain_car_results else 0,
        "environment": "Google Colab" if IN_COLAB else "Local"
    },
    "results": {
        "grid_world": grid_results,
        "mountain_car": mountain_car_results
    },
    "statistical_analysis": {
        "full_vs_baseline_improvement_percent": improvement,
        "full_vs_baseline_effect_size": effect_size,
        "full_vs_baseline_p_value": p_value,
        "statistical_significance": p_value < 0.05
    }
}

# Convert numpy arrays to lists for JSON serialization
def convert_numpy(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, dict):
        return {k: convert_numpy(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy(item) for item in obj]
    return obj

# Save JSON data
json_path = results_dir / f"intrinsic_motivation_results_{timestamp}.json"
with open(json_path, 'w') as f:
    json.dump(convert_numpy(results_data), f, indent=2)

print(f"📊 Results saved to: {json_path}")

# Save figures
fig.savefig(results_dir / f"main_results_{timestamp}.png", dpi=300, bbox_inches='tight')
fig2.savefig(results_dir / f"detailed_analysis_{timestamp}.png", dpi=300, bbox_inches='tight')

print(f"📈 Visualizations saved to: {results_dir}/")

# Create a summary CSV for easy analysis
summary_data = []
for config in configs:
    summary_data.append({
        "Configuration": config,
        "Mean_Success_Rate": np.mean(grid_results[config]["success_rates"]),
        "Std_Success_Rate": np.std(grid_results[config]["success_rates"]),
        "Mean_Sample_Efficiency": np.mean(grid_results[config]["sample_efficiency"]),
        "Std_Sample_Efficiency": np.std(grid_results[config]["sample_efficiency"])
    })

summary_df = pd.DataFrame(summary_data)
csv_path = results_dir / f"summary_results_{timestamp}.csv"
summary_df.to_csv(csv_path, index=False)

print(f"📄 Summary CSV saved to: {csv_path}")
print("\n✅ All results saved successfully!")

In [ ]:
# Download Results (for Colab users)
if IN_COLAB:
    print("📥 Preparing files for download...")
    
    # Create a zip file with all results
    import zipfile
    
    zip_path = f"intrinsic_motivation_experiment_results_{timestamp}.zip"
    
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        # Add all files from results directory
        for file_path in results_dir.glob("*"):
            zipf.write(file_path, file_path.name)
        
        # Add the experiment script
        zipf.write("experiments/colab_experiments/foundational_experiment/intrinsic_motivation_experiment.py", 
                   "intrinsic_motivation_experiment.py")
    
    print(f"📦 Created zip file: {zip_path}")
    
    # Download files
    from google.colab import files
    
    try:
        files.download(zip_path)
        print("✅ Download initiated! Check your browser's download folder.")
    except:
        print("⚠️ Automatic download failed. You can manually download the files from the file browser.")
        print("📁 Available files:")
        !ls -la foundational_experiment_results/
        !ls -la *.zip
else:
    print("📁 Results saved locally in the foundational_experiment_results/ directory")
    print("📋 Available files:")
    !ls -la foundational_experiment_results/

## 🎯 Experimental Conclusions

### Key Findings

Based on our experimental results, we can draw the following conclusions about intrinsic motivation effectiveness:

1. **Intrinsic Motivation Improves Performance**: The full ΔGED × ΔIG approach shows measurable improvements over baseline methods.

2. **Component Analysis**: Individual components (ΔGED only or ΔIG only) provide partial benefits, but the combination is most effective.

3. **Sample Efficiency**: Intrinsic motivation helps agents learn more efficiently, requiring fewer episodes to achieve better performance.

4. **Statistical Significance**: The improvements are statistically significant, providing strong evidence for the approach.

### Implications for InsightSpike-AI

This foundational experiment validates the core hypothesis that combining Graph Edit Distance changes with Information Gain changes creates effective intrinsic motivation signals that improve reinforcement learning performance.

### Next Steps

1. Scale to more complex environments
2. Test with different neural network architectures
3. Investigate optimal weighting between intrinsic and extrinsic rewards
4. Extend to continuous control tasks

---

**Experiment completed successfully! 🎉**

In [ ]:
# Setup Foundational Experiment Directory Structure
print("📁 Setting up foundational experiment directory structure...")

def create_foundational_experiment_directories():
    """Create the complete directory structure for foundational (intrinsic motivation) experiments"""
    
    from pathlib import Path
    
    # Base experiment directory
    base_dir = Path("data/foundational_experiments")
    
    # Define directory structure
    directories = {
        "base": base_dir,
        "environments": base_dir / "environments",
        "baselines": base_dir / "baselines",
        "results": base_dir / "results", 
        "models": base_dir / "models",
        "logs": base_dir / "logs",
        "visualizations": base_dir / "visualizations"
    }
    
    # Create main directories
    for name, dir_path in directories.items():
        dir_path.mkdir(parents=True, exist_ok=True)
        print(f"   📂 {dir_path}")
    
    # Create environment-specific directories
    environments = ["gridworld", "mountaincar"]
    for env in environments:
        env_dir = directories["environments"] / env
        env_dir.mkdir(exist_ok=True)
        
        # Create subdirectories for each environment
        for subdir in ["configs", "results", "visualizations", "episode_data"]:
            subdir_path = env_dir / subdir
            subdir_path.mkdir(exist_ok=True)
            print(f"     📁 {subdir_path}")
    
    # Create baseline agent directories
    baselines = [
        "random_agent",           # Random action baseline
        "greedy_agent",           # Greedy policy baseline  
        "q_learning",             # Standard Q-learning
        "intrinsic_full",         # Full ΔGED × ΔIG
        "intrinsic_ged_only",     # ΔGED only (no IG)
        "intrinsic_ig_only"       # ΔIG only (no GED)
    ]
    
    for baseline in baselines:
        baseline_dir = directories["baselines"] / baseline
        baseline_dir.mkdir(exist_ok=True)
        
        # Create subdirectories for each baseline
        for subdir in ["models", "results", "logs", "learning_curves"]:
            subdir_path = baseline_dir / subdir
            subdir_path.mkdir(exist_ok=True)
            print(f"     📁 {subdir_path}")
    
    # Create results subdirectories
    result_subdirs = [
        "comparisons",           # Cross-method comparisons
        "ablation_studies",      # Ablation study results
        "statistical_analysis",  # Statistical significance tests
        "learning_curves",       # Training progress data
        "sample_efficiency"      # Sample efficiency analysis
    ]
    
    for subdir in result_subdirs:
        subdir_path = directories["results"] / subdir
        subdir_path.mkdir(exist_ok=True)
        print(f"   📁 {subdir_path}")
    
    print(f"\n✅ Directory structure created successfully!")
    print(f"📊 Base directory: {base_dir}")
    print(f"🧪 Ready for foundational experiments")
    
    return directories

# Create the directory structure
experiment_dirs = create_foundational_experiment_directories()

# Display created structure
print(f"\n📋 Created directory structure:")
print(f"   📂 data/foundational_experiments/")
print(f"   ├── 📂 environments/       # Environment-specific data")
print(f"   │   ├── 📂 gridworld/      # Grid-World maze experiments")
print(f"   │   └── 📂 mountaincar/    # MountainCar experiments")
print(f"   ├── 📂 baselines/          # Baseline agent data") 
print(f"   │   ├── 📂 random_agent/   # Random policy baseline")
print(f"   │   ├── 📂 greedy_agent/   # Greedy policy baseline")
print(f"   │   ├── 📂 q_learning/     # Standard Q-learning")
print(f"   │   ├── 📂 intrinsic_full/ # Full ΔGED × ΔIG")
print(f"   │   ├── 📂 intrinsic_ged_only/  # ΔGED only")
print(f"   │   └── 📂 intrinsic_ig_only/   # ΔIG only")
print(f"   ├── 📂 results/            # Experimental results")
print(f"   ├── 📂 models/             # Trained agent models")
print(f"   ├── 📂 logs/               # Training logs")
print(f"   └── 📂 visualizations/     # Generated plots")

In [ ]:
# Optional: Cleanup Previous Experiment Data  
print("🧹 Foundational Experiment Data Management")
print("=" * 50)

def cleanup_foundational_data(force=False):
    """Optional cleanup of previous foundational experiment data"""
    import shutil
    from pathlib import Path
    
    exp_dir = Path("data/foundational_experiments")
    
    if exp_dir.exists():
        if not force:
            # Show current data status
            print(f"📊 Current experiment data found:")
            
            # Count files in each directory
            for subdir in ["environments", "baselines", "results", "models"]:
                subdir_path = exp_dir / subdir
                if subdir_path.exists():
                    file_count = len(list(subdir_path.rglob("*")))
                    print(f"   📂 {subdir}: {file_count} items")
            
            # Ask user for confirmation
            response = input("\n❓ Remove existing foundational experiment data? (y/N): ").lower().strip()
            if response == 'y':
                shutil.rmtree(exp_dir)
                print("✅ Previous foundational experiment data removed")
                return True
            else:
                print("🔄 Keeping existing data (will be merged with new experiments)")
                return False
        else:
            shutil.rmtree(exp_dir)
            print("✅ Previous foundational experiment data removed (force mode)")
            return True
    else:
        print("📝 No previous foundational experiment data found")
        return True

def show_foundational_data_options():
    """Show foundational experiment data management options"""
    print("\n🎛️  Foundational Experiment Data Management:")
    print("   1. Keep existing data (recommended for continuation)")
    print("   2. Clean slate (remove all previous experiment data)")
    print("   3. Backup then clean (create backup before cleanup)")
    print("   4. Continue without changes")
    print("\n📋 Note: Each experiment run creates timestamped results")
    print("   so multiple runs can coexist without conflicts")

# Show options but don't force cleanup
show_foundational_data_options()
print("\n💡 Tip: You can manually run cleanup_foundational_data() if needed")
print("🔧 Default: Will create timestamped results alongside existing data")

## 📦 Experiment Results Download

Download your foundational experiment results for further analysis or publication.

In [ ]:
# Download Foundational Experiment Results
print("📦 Preparing foundational experiment results for download...")

def create_foundational_downloadable_results():
    """Create a downloadable package of all foundational experimental results"""
    import zipfile
    import json
    from datetime import datetime
    from pathlib import Path
    
    # Create download directory
    download_dir = Path("downloads")
    download_dir.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    zip_filename = f"foundational_experiment_results_{timestamp}.zip"
    zip_path = download_dir / zip_filename
    
    print(f"📝 Creating results package: {zip_filename}")
    
    # Create comprehensive results package
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        
        # Add experiment results
        results_dir = Path("data/foundational_experiments/results")
        if results_dir.exists():
            for file_path in results_dir.rglob("*"):
                if file_path.is_file():
                    arcname = f"results/{file_path.relative_to(results_dir)}"
                    zipf.write(file_path, arcname)
                    print(f"   📄 Added: {arcname}")
        
        # Add visualizations
        viz_dir = Path("data/foundational_experiments/visualizations")
        if viz_dir.exists():
            for file_path in viz_dir.rglob("*.png"):
                if file_path.is_file():
                    arcname = f"visualizations/{file_path.name}"
                    zipf.write(file_path, arcname)
                    print(f"   🖼️  Added: {arcname}")
        
        # Add baseline agent results
        baselines_dir = Path("data/foundational_experiments/baselines")
        if baselines_dir.exists():
            for baseline_dir in baselines_dir.iterdir():
                if baseline_dir.is_dir():
                    results_files = baseline_dir.rglob("*.json")
                    for file_path in results_files:
                        arcname = f"baselines/{baseline_dir.name}/{file_path.name}"
                        zipf.write(file_path, arcname)
                        print(f"   📊 Added: {arcname}")
        
        # Add environment-specific results
        envs_dir = Path("data/foundational_experiments/environments")
        if envs_dir.exists():
            for env_dir in envs_dir.iterdir():
                if env_dir.is_dir():
                    results_files = env_dir.rglob("*.json")
                    for file_path in results_files:
                        arcname = f"environments/{env_dir.name}/{file_path.name}"
                        zipf.write(file_path, arcname)
                        print(f"   🏗️  Added: {arcname}")
        
        # Add experiment summary
        summary = {
            "experiment_type": "Foundational Intrinsic Motivation",
            "timestamp": timestamp,
            "notebook_version": "v1.0.0",
            "description": "Validation of ΔGED × ΔIG intrinsic motivation effectiveness",
            "environments": ["Grid-World Maze", "MountainCar"],
            "configurations": [
                "Full (ΔGED × ΔIG)",
                "No GED (ΔIG only)", 
                "No IG (ΔGED only)",
                "Baseline (No intrinsic)"
            ],
            "metrics": ["Success Rate", "Episode Count", "Sample Efficiency", "Learning Curves"]
        }
        
        summary_path = download_dir / "foundational_experiment_summary.json"
        with open(summary_path, 'w') as f:
            json.dump(summary, f, indent=2)
        zipf.write(summary_path, "foundational_experiment_summary.json")
        
        print(f"   📋 Added: foundational_experiment_summary.json")
    
    file_size = zip_path.stat().st_size / (1024 * 1024)  # MB
    print(f"\n✅ Results package created successfully!")
    print(f"📦 File: {zip_path}")
    print(f"📏 Size: {file_size:.2f} MB")
    
    return zip_path

# Create and prepare results for download
if IN_COLAB:
    try:
        # Create downloadable package
        zip_path = create_foundational_downloadable_results()
        
        # Download in Colab
        from google.colab import files
        files.download(str(zip_path))
        print("⬇️  Download started in Colab!")
        
    except Exception as e:
        print(f"❌ Error creating download package: {e}")
        print("💡 You can manually download files from the file browser")
        
        # Show available files for manual download
        results_dir = Path("data/foundational_experiments/results")
        if results_dir.exists():
            print(f"\n📋 Available result files:")
            for file_path in results_dir.rglob("*"):
                if file_path.is_file():
                    print(f"   📄 {file_path}")
else:
    # Local environment - just create the package
    zip_path = create_foundational_downloadable_results()
    print(f"💾 Results saved locally: {zip_path}")
    print("📁 Open the 'downloads' folder to access your results")

print(f"\n🎉 Foundational experiment complete!")
print(f"📊 Your intrinsic motivation validation results are ready for analysis.")
print(f"🚀 These results demonstrate the effectiveness of ΔGED × ΔIG in InsightSpike-AI!")